**Import libraries**

In [ ]:
from torchvision.datasets import VisionDataset
import os
import os.path
import sys
import logging
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import torch
from sklearn.utils import shuffle
import torch.nn as nn
import torch.optim as optim# Create a SummaryWriter instance
from torch.utils.data import Subset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.backends import cudnn
import torchvision
from torchvision import transforms
from torchvision.models import alexnet, resnet18, resnet34
from PIL import Image
from tqdm import tqdm
import tensorflow as tf  # 仅用于兼容性
import tensorboard as tb

**Set Arguments**

In [2]:
DEVICE = 'cuda' # 'cuda' or 'cpu'

NUM_CLASSES = 101    # 101 + 1: There is am extra Background class that should be removed 

BATCH_SIZE = 256    # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 0.01         # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5 # Regularization, you can keep this at the default
NUM_EPOCHS = 30      # Total number of training epochs (iterations over dataset)

STEP_SIZE = 10      # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.5         # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

**Define Data Preprocessing**

In [3]:
# Define transforms for training phase
train_transform = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      #transforms.RandomCrop(size=150),
                                      #transforms.RandomHorizontalFlip(0.5),
                                      #transforms.ColorJitter(brightness=0.3, contrast=0.5, saturation=0.3),
                                      #transforms.RandomRotation(degrees=45, center=(0,0)),
                                      #transforms.RandomRotation(degrees=45),
                                      #transforms.RandomGrayscale(0.5),
                                      #transforms.RandomErasing(p=0.5, scale=(0.02, 0.33)),
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))#Normalizes tensor for pretrained net on ImageNet
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizes tensor with mean and standard deviation
                                      

                                      
])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))#Normalizes tensor for pretrained net on ImageNet
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  
                                      

])

**Prepare Dataset**

In [4]:

DATA_DIR = '/kaggle/input/caltech101/Caltech101/101_ObjectCategories'
#from Caltech101.caltech_dataset import Caltech

In [5]:
def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')


class Caltech(VisionDataset):
    def __init__(self, root, split='train', transform=None, target_transform=None):
        super(Caltech, self).__init__(root, transform=transform, target_transform=target_transform)
        
        self.split = split+".txt" # This defines the split you are going to use
                                         
        self.images=[]
        classes=os.listdir(root)
        classes.remove('BACKGROUND_Google')
        classes.sort()
        #create dictionary
        n=0
        self.Dict={}
        for s in classes:
          self.Dict[s]=n
          n=n+1

        file = open("/kaggle/input/caltech101/Caltech101/"+self.split)
        for l in file:
            line=l.split("/")
            classe=line[0]
            if classe!='BACKGROUND_Google':
              name=line[1].replace("\n","")
              self.images.append(classe+"/"+name)
        file.close()

    def __getitem__(self, index):
      string=self.images[index].split("/")
      classe=string[0]
      image, label = pil_loader("/kaggle/input/caltech101/Caltech101/101_ObjectCategories/"+self.images[index]), self.Dict[classe]
      # Applies preprocessing when accessing the image
      if self.transform is not None:
        image = self.transform(image)
 
        return image, label

    def __len__(self):
        length = len(self.images) # Provide a way to get the length (number of elements) of the dataset
        return length

In [6]:
# Prepare Pytorch train/test Datasets

train_dataset_pre = Caltech(DATA_DIR, split='train',  transform=train_transform)
test_dataset = Caltech(DATA_DIR, split='test', transform=eval_transform)

#della stessa dimensinoe e visto che le classi sono in ordine anche stratificato
train_indexes = [x for x in range(len(train_dataset_pre)) if not(x % 2) == 0]
val_indexes = [x for x in range(len(train_dataset_pre)) if x % 2 == 0]

train_dataset = Subset(train_dataset_pre, train_indexes)
val_dataset = Subset(train_dataset_pre, val_indexes)

# Check dataset sizes
print('Train Dataset: {}'.format(len(train_dataset)))
print('Valid Dataset: {}'.format(len(val_dataset)))
print('Test Dataset: {}'.format(len(test_dataset)))

Train Dataset: 2892
Valid Dataset: 2892
Test Dataset: 2893


In [7]:
train_dataset_pre

Dataset Caltech
    Number of datapoints: 5784
    Root location: /kaggle/input/caltech101/Caltech101/101_ObjectCategories
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=True)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

**Prepare Dataloaders**

In [8]:
# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

**Prepare Network**

In [ ]:
#net=resnet18(pretrained=True)
#net=resnet34(pretrained=True)
#num_ft = net.fc.in_features
#net.fc = nn.Linear(num_ft, NUM_CLASSES)

#net = alexnet() # Loading AlexNet model
net=alexnet(pretrained=True) #Load pretrined AlexNet on ImageNet

# AlexNet has 1000 output neurons, corresponding to the 1000 ImageNet's classes
# We need 101 outputs for Caltech-101
net.classifier[6] = nn.Linear(4096, NUM_CLASSES) # nn.Linear in pytorch is a fully connected layer
                                                 # The convolutional layer is nn.Conv2d

# We just changed the last layer of AlexNet with a new fully connected layer with 101 outputs



######### freeze all the conv and train the fc
for layer in net.features.parameters():
    layer.requires_grad=False
################################################
################freeze all the fc 
# for layer in net.classifier.parameters():
#  layer.requires_grad=False
###############################################

#Note that you can filter the parameters so that only the parameters that requires gradient are passed to the optimizer.
for layer in net.parameters():
    print(layer.requires_grad)
#ft_list = list(net.features)
#cl_list = list(net.classifier)
#cl_list

**Prepare Training**

In [10]:
# Define loss function
criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

# Choose parameters to optimize
parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet

# Define optimizer
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

# Define scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

**Train**

In [ ]:
# By default, everything is loaded to cpu
net = net.to(DEVICE)  # this will bring the network to GPU if DEVICE is cuda
cudnn.benchmark  # Calling this optimizes runtime
#######################
# Keep the accuracy result 
myacc = []
# Keep the loss
myloss = []
# Keep the LR
mylr = []
#######################
best_net = net
max_acc = 0
current_step = 0

from torch.utils.tensorboard import SummaryWriter




# 初始化 TensorBoard 写入器
writer = SummaryWriter('runs/cnn_caltech101')

# 定义保存模型的路径
save_path = '/kaggle/working/best_model.pth'

# 在训练循环中记录数据
for epoch in range(NUM_EPOCHS):
    print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_last_lr()))
    mylr.append(scheduler.get_last_lr())
    net.train(True)
    running_loss = 0.0
    running_corrects_train = 0  # To keep track of training accuracy
    
    for images, labels in train_dataloader:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        current_step += 1
        
        # 计算训练集准确率
        _, preds = torch.max(outputs.data, 1)
        running_corrects_train += torch.sum(preds == labels.data).data.item()

    # 记录训练集损失和准确率
    train_loss = running_loss / len(train_dataloader)
    train_acc = running_corrects_train / float(len(train_dataset))
    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Accuracy/train', train_acc, epoch)

    # 验证
    if epoch % 5 == 0:
        net.train(False)
        running_corrects_val = 0
        running_loss_val = 0.0  # 用于验证集的损失
        
        for images, labels in tqdm(val_dataloader):
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)
            outputs = net(images)
            loss = criterion(outputs, labels)
            running_loss_val += loss.item()
            
            # 计算验证集准确率
            _, preds = torch.max(outputs.data, 1)
            running_corrects_val += torch.sum(preds == labels.data).data.item()

        # 计算验证集损失和准确率
        val_loss = running_loss_val / len(val_dataloader)
        val_acc = running_corrects_val / float(len(val_dataset))
        writer.add_scalar('Loss/val', val_loss, epoch)
        writer.add_scalar('Accuracy/val', val_acc, epoch)

        if val_acc >= max_acc:
            max_acc = val_acc
            best_model = net
            # 保存最佳模型的权重和参数
            #torch.save(best_model.state_dict(), save_path)  # 保存模型

        print('Validation Accuracy: {}'.format(val_acc) + ' at epoch: ' + str(epoch+1))
    
    # Step the scheduler
    scheduler.step()

# 关闭写入器
writer.close()

# 启动 TensorBoard
%load_ext tensorboard
%tensorboard --logdir runs/cnn_caltech101



In [ ]:
net=best_net
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in tqdm(test_dataloader):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))

print('Test Accuracy: {}'.format(accuracy))